# Models with External Subsystems

In level 2, we have given simple examples of defining external subsystems in `phase_info`. The subsystems that we gave were all dummy subsystems and are not really used in simulation. Assume you have an external subsystem and you want to use it. Let us how to achieve this goal.

## Installation of Aviary examples

The Aviary team has provided a few external subsystems for you to use.
These are included in the `aviary/examples/external_subsystems` directory.
We'll now discuss them here and show how to use them.

## Adding simple_weight subsystems

Currently, there are a couple of examples: `battery` and `simple_weight`. Let us take a look at `simple_weight` first. As shown in this example, this is a simplified example of a component that computes a weight for the wing and horizontal tail. It does not provide realistic computations but rough estimates to `Aircraft.Wing.MASS` and `Aircraft.HorizontalTail.MASS`. When this external subsystem is added to your pre-mission phase, Aviary will compute these weights in its core subsystem as usual, but then the wing mass and tail mass values will be overridden by this external subsytem.

In [level 2](onboarding_level2), we have briefly covered how to add external subsystem in `phase_info`. Alternatively, external subsystems (and any other new keys) can be added after a `phase_info` is loaded. Let us see how it works using the aircraft_for_bench_FwFm.csv model. First, we import this particular external subsystem.

Then add this external subsystem to `pre_mission`.
That is all you need to do in addition to our traditional level 2 examples. Here is the complete run script,

In [ ]:
from copy import deepcopy
from aviary.api import Aircraft
import aviary.api as av

from aviary.examples.external_subsystems.simple_weight.simple_weight_builder import WingWeightBuilder


phase_info = deepcopy(av.default_simple_phase_info)
# Here we just add the simple weight system to only the pre-mission
phase_info['pre_mission']['external_subsystems'] = [WingWeightBuilder(name="wing_external")]

prob = av.AviaryProblem(phase_info, mission_method="simple", mass_method="FLOPS")

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs('models/test_aircraft/aircraft_for_bench_FwFm.csv')

# Have checks for clashing user inputs
# Raise warnings or errors depending on how clashing the issues are
# prob.check_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver("SLSQP")

prob.add_design_variables()

prob.add_objective()

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(suppress_solver_print=True)

print('Engine Mass', prob.get_val(av.Aircraft.Engine.MASS))
print('Wing Mass', prob.get_val(av.Aircraft.Wing.MASS))
print('Horizontal Tail Mass', prob.get_val(av.Aircraft.HorizontalTail.MASS))

Ignore the intermediate warning messages and you see the outputs at the end.
Since this is a `FLOPS` mission and no objective is provided, we know that the objective is `fuel_burned`.

To see the outputs without external subsystem add-on, let us comment out the lines that add the wing weight builder and run the modified script:

In [ ]:
# # Here we just add the simple weight system to only the pre-mission
# phase_info['pre_mission']['external_subsystems'] = [WingWeightBuilder(name="wing_external")]

prob = av.AviaryProblem(av.default_simple_phase_info, mission_method="simple", mass_method="FLOPS")

# Load aircraft and options data from user
# Allow for user overrides here
prob.load_inputs('models/test_aircraft/aircraft_for_bench_FwFm.csv')

# Have checks for clashing user inputs
# Raise warnings or errors depending on how clashing the issues are
# prob.check_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver("SLSQP")

prob.add_design_variables()

prob.add_objective()

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem(suppress_solver_print=True)

print('Engine Mass', prob.get_val(Aircraft.Engine.MASS))
print('Wing Mass', prob.get_val(Aircraft.Wing.MASS))
print('Horizontal Tail Mass', prob.get_val(Aircraft.HorizontalTail.MASS))

As we see, the engine mass is not altered but wing mass and tail mass are changed dramatically. This is not surprising because our `simple_weight` subsystem is quite simple. Later on, we will show you a more realistic wing weight external subsystem.

## Adding battery subsystem

In the above example, there is no new Aviary variable added to Aviary and the external subsystem is added to pre-mission only. So, the subsystem is not very involved. We will see a more complicated example now. Before we move on, let us recall the steps in Aviary model building:

- **init**
- **load_inputs**
- **check_inputs**
- **add_pre_mission_systems**
- **add_phases**
- **add_post_mission_systems**
- **link_phases**
- add_driver
- **add_design_variables**
- **add_objective**
- setup
- **set_initial_guesses**
- run_aviary_problem

The steps in bold are related specifically to subsystems. So, almost all of the steps involve subsystems. As long as your external subsystem is built based on the guidelines, Aviary will take care your subsystem.

The next example is the [battery subsystem](https://github.com/OpenMDAO/Aviary/blob/main/aviary/docs/user_guide/battery_subsystem_example.md). The battery subsystem provides methods to define the battery subsystem's states, design variables, fixed values, initial guesses, and mass names. It also provides methods to build OpenMDAO systems for the pre-mission and mission computations of the subsystem, to get the constraints for the subsystem, and to preprocess the inputs for the subsystem. This subsystem has its own set of variables. We will build an Aviary model with full phases (namely, `climb`, `cruise` and `descent`) and maximize the final total mass: `Dynamic.Mission.MASS`.

We also need `BatteryBuilder` along with battery related aircraft variables and build a new battery object.
Now, add our new battery subsystem into each phase including pre-mission:


In [ ]:
from aviary.examples.external_subsystems.battery.battery_builder import BatteryBuilder
from aviary.examples.external_subsystems.battery.battery_variables import Aircraft

battery_builder = BatteryBuilder(include_constraints=False)

phase_info['pre_mission']['external_subsystems'] = [battery_builder]
phase_info['climb']['external_subsystems'] = [battery_builder]
phase_info['cruise']['external_subsystems'] = [battery_builder]
phase_info['descent']['external_subsystems'] = [battery_builder]

Start an Aviary problem and load in aircraft input deck:

In [ ]:
prob = av.AviaryProblem(phase_info, mission_method="simple", mass_method="FLOPS")

prob.load_inputs('models/test_aircraft/aircraft_for_bench_FwFm.csv')


In the battery subsystem, the type of battery cell we use is `18650`. The option is not set in `input_file`, instead is is controlled by importing the correct battery map [here](https://github.com/OpenMDAO/om-Aviary/blob/1fca1c03cb2e1d6387442162e8d7dabf83eee197/aviary/examples/external_subsystems/battery/model/reg_thevenin_interp_group.py#L5).

We can then check our inputs:

In [ ]:
# Have checks for clashing user inputs
# Raise warnings or errors depending on severity of the issues
prob.check_inputs()

### Checking in setup function call

Function `setup` can have an argument `check` with default value `False`. If we set it to `True`, there will cause a default set of checks to run. So, instead of a simple `prob.setup()` call, let us do the following:

The following are a few check points printed on the command line:

```
INFO: checking system
INFO: checking solvers
INFO: checking dup_inputs
INFO: checking missing_recorders
```



In [ ]:
prob.add_pre_mission_systems()

traj = prob.add_phases()

prob.add_post_mission_systems()

# Link phases and variables
prob.link_phases()

prob.add_driver("SLSQP")

prob.add_design_variables()

prob.add_objective('mass')

prob.setup(check=True)

prob.set_initial_guesses()

prob.run_aviary_problem()

# user defined outputs
print('Battery MASS', prob.get_val(Aircraft.Battery.MASS))
print('Cell Max', prob.get_val(Aircraft.Battery.Cell.MASS))
masses_descent = prob.get_val('traj.descent.timeseries.states:mass', units='kg')
print(f"Final Descent Mass: {masses_descent[-1]}")

print('done')

## More on outputs

We are done with our model. For our current example, let us add a few more lines after aviary run:

In [ ]:
print('Battery MASS', prob.get_val(Aircraft.Battery.MASS))
print('Cell Max', prob.get_val(Aircraft.Battery.Cell.MASS))

Since our objective is `mass`, we want to print the value of `Dynamic.Mission.Mass`. Remember, we have imported Dynamic from aviary.variable_info.variables for this purpose.

So, we have to print the final mass in a different way. Keep in mind that we have three phases in the mission and that final mass is our objective. So, we can get the final mass of descent phase instead. Let us try this approach. Let us comment out the print statement of final mass (and the import of Dynamic), then add the following lines:

In [ ]:
masses_descent = prob.get_val('traj.descent.timeseries.states:mass', units='kg')
print(f"Final Descent Mass: {masses_descent[-1]}")

## More on objectives

Now, let us change our objective to battery state of charge after climb phase. So, comment out `prob.add_objective('mass')` and add the following line right after:

```
prob.model.add_objective(
    f'traj.climb.states:{Mission.Battery.STATE_OF_CHARGE}', index=-1, ref=-1)
```

In the above, `index=-1` means the end of climb phase and `ref=-1` means that we want to maximize the state of charge at the end of climb phase. Once again, we are unable to print battery state of charge as we did with battery mass and battery cell mass. We will use the trick to get mass. In fact, we have prepared for this purpose by setting up time series of climb and cruise phases as well. All we need to do is to add the following lines:

```
soc_cruise = prob.get_val(
    'traj.climb.timeseries.states:mission:battery:state_of_charge')
print(f"State of Charge: {soc_cruise[-1]}")
```

Now you get a new output:

```
State of Charge: [0.97458496]

## The check_partials function

In order to make sure that your model computes all the derivatives correctly, OpenMDAO provides a method called `check_partials` which checks partial derivatives comprehensively for all Components in your model. Let us continue in our model:

In [ ]:
prob.check_partials(compact_print=True)

## Adding an OpenAeroStruct wingbox external subsystem

[OpenAeroStruct](https://github.com/mdolab/OpenAerostruct/) (OAS) is a lightweight tool that performs aerostructural optimization using OpenMDAO.

### Installation of OpenAeroStruct

We would like to have easy access to the examples and source code. So we install OpenAeroStruct by cloning the OpenAeroStruct repository. We show you how to do the installation on Linux. Assume you want to install it at `~/$USER/workspace`. Do the following:

```
cd ~/$USER/workspace
git clone https://github.com/mdolab/OpenAeroStruct.git
~/$USER/workspace/OpenAeroStruct
pip install -e .
```

If everything runs smoothly, you should see something like:

```
Successfully installed openaerostruct
```

Most of the packages that OpenAeroStruct depends on are installed already (see [here](https://mdolab-openaerostruct.readthedocs-hosted.com/en/latest/installation.html)). For our example, we need [ambiance](https://pypi.org/project/ambiance/) and an optional package: [OpenVSP](http://openvsp.org/).

To install `ambiance`, do the following:

```
pip install ambiance
```

You should see something like:

```
Installing collected packages: ambiance
Successfully installed ambiance-1.3.1
```

```{note}
You must ensure that the Python version in your environment matches the Python used to compile OpenVSP. You must [install OpenVSP](https://openvsp.org/wiki/doku.php?id=install) on your Linux box yourself.
```

To check your installation of OpenVSP is successful, please run

```
(av1)$ python openaerostruct/tests/test_vsp_aero_analysis.py
```

Windows users should visit [OpenVSP](http://openvsp.org/download.php) and follow the instruction there.